In [1]:
import sys
import os

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.abspath('')
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

In [98]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import re
import pickle

import importlib
import utils
importlib.reload(utils)
from tqdm import tqdm

from utils import get_parsed, get_parsed_remove_comment, baseball_base_url, ordinal, clean_name

In [4]:
df_all_leaders = pd.read_csv('./all_leaders.csv')

In [5]:
df_all_leaders

,Statistic,Single-Season,Career,Active,Progressive,Yearly League,Year-by-Year Top-Tens,Category
0,Wins Above Replacement,/leaders/WAR_season.shtml,/leaders/WAR_career.shtml,/leaders/WAR_active.shtml,/leaders/WAR_progress.shtml,/leaders/WAR_leagues.shtml,/leaders/WAR_top_ten.shtml,Player Value Leaderboards
1,WAR Position Players,/leaders/WAR_bat_season.shtml,/leaders/WAR_bat_career.shtml,/leaders/WAR_bat_active.shtml,/leaders/WAR_bat_progress.shtml,/leaders/WAR_bat_leagues.shtml,/leaders/WAR_bat_top_ten.shtml,Batting Leaderboards
2,Offensive WAR,/leaders/WAR_off_season.shtml,/leaders/WAR_off_career.shtml,/leaders/WAR_off_active.shtml,/leaders/WAR_off_progress.shtml,/leaders/WAR_off_leagues.shtml,/leaders/WAR_off_top_ten.shtml,Batting Leaderboards
3,Defensive WAR,/leaders/WAR_def_season.shtml,/leaders/WAR_def_career.shtml,/leaders/WAR_def_active.shtml,/leaders/WAR_def_progress.shtml,/leaders/WAR_def_leagues.shtml,/leaders/WAR_def_top_ten.shtml,Batting Leaderboards
4,Batting Average,/leaders/batting_avg_season.shtml,/leaders/batting_avg_career.shtml,/leaders/batting_avg_active.shtml,/leaders/batting_avg_progress.shtml,/leaders/batting_avg_leagues.shtml,/leaders/batting_avg_top_ten.shtml,Batting Leaderboards
...,...,...,...,...,...,...,...,...
174,Fielding % as P,/leaders/fielding_perc_p_season.shtml,/leaders/fielding_perc_p_career.shtml,/leaders/fielding_perc_p_active.shtml,/leaders/fielding_perc_p_progress.shtml,/leaders/fielding_perc_p_leagues.shtml,/leaders/fielding_perc_p_top_ten.shtml,Fielding Leaderboards
175,Replay Challenges,/leaders/mgr_challenge_count_season.shtml,/leaders/mgr_challenge_count_career.shtml,/leaders/mgr_challenge_count_active.shtml,/leaders/mgr_challenge_count_progress.shtml,/leaders/mgr_challenge_count_leagues.shtml,/leaders/mgr_challenge_count_top_ten.shtml,Manager Leaderboards
176,Successful Replay Challenges,/leaders/mgr_overturn_count_season.shtml,/leaders/mgr_overturn_count_career.shtml,/leaders/mgr_overturn_count_active.shtml,/leaders/mgr_overturn_count_progress.shtml,/leaders/mgr_overturn_count_leagues.shtml,/leaders/mgr_overturn_count_top_ten.shtml,Manager Leaderboards
177,Replay Challenge Success %,/leaders/mgr_replay_success_rate_season.shtml,/leaders/mgr_replay_success_rate_career.shtml,/leaders/mgr_replay_success_rate_active.shtml,/leaders/mgr_replay_success_rate_progress.shtml,/leaders/mgr_replay_success_rate_leagues.shtml,/leaders/mgr_replay_success_rate_top_ten.shtml,Manager Leaderboards


In [6]:
df_single_season = df_all_leaders[['Statistic', 'Single-Season', 'Category']]
df_single_season.head(2)

,Statistic,Single-Season,Category
0,Wins Above Replacement,/leaders/WAR_season.shtml,Player Value Leaderboards
1,WAR Position Players,/leaders/WAR_bat_season.shtml,Batting Leaderboards


In [8]:
def process_statistic(table):
    def process_row(row):
        res = []
        for td in row.find_all('td'):
            res.append(re.sub('\xa0', ' ', td.text.strip()))
        #print(res)
        return res
        pass
    
    columns_name = [i.text.strip() for i in table.find('thead').find_all('th')]
    #print(columns_name)
    
    data = []
    for row in table.find_all('tr')[1:]:
        #print(row)
        processed_row = process_row(row)
        if not processed_row:
            continue
        if processed_row[0] == '':
            processed_row[0] = data[-1][0]
        data.append(processed_row)
    
    df = pd.DataFrame(data, columns=columns_name)
    
    return df

In [19]:
all_single_season  = []
for i in tqdm(range(min(df_single_season.shape[0], 20000))):
    
    content = get_parsed(baseball_base_url + df_single_season.loc[i, 'Single-Season'])
    table = content.find('table')
    
    df = process_statistic(table)
    
    df['Rank'] = df['Rank'].map(lambda x: x[:-1]).astype(int)
    cat = df_single_season.loc[i, 'Category']
    cat = " ".join(cat.split(" ")[:-1])
    all_single_season.append([(cat, df_single_season.loc[i, 'Statistic']), df])

100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [01:07<00:00,  2.65it/s]


In [20]:
with open('pkl_leaders_single_season.pkl', 'wb') as f:
    pickle.dump(all_single_season, f)

In [27]:
test_id = 100
test_df = all_single_season[test_id][1]
all_single_season[test_id][0]

('Fielding', 'Assists as 2B')

In [28]:
test_df[test_df['Rank'] == 1]

,Rank,Player (age that year),Assists as 2B,Year,Throws
0,1,Frankie Frisch+ (29),641,1927,R


# Generate questions

In [107]:
wrong_count = 4
columns = ['Question', 'Correct'] + ['Wrong_' + str(i + 1) for i in range(wrong_count)] +  ['tags']
ranking_tag = 'Single-season Ranking'
leader_name = 'single-season'
question_list = []

import random
random.seed(1000)
for (cat, stat), df in tqdm(all_single_season[:min(len(all_single_season), 1000)]):
    name_column = df.columns[1]
    stat_column = df.columns[2]
    ranks_unique = df['Rank'].unique()
    #print(stat)
    #print(len(ranks_unique))
    small_range = 6
    middle_range = int(small_range * 1.8)
    large_range = int(middle_range * 1.8)
    small_range_rank = list(ranks_unique[:small_range])
    middle_range_rank = list(ranks_unique[:middle_range])
    large_range_rank = list(ranks_unique[:large_range])
    
    if len(ranks_unique) < 10:
        continue
    # who is number 1st - ith
    for i in range(small_range):
        rank = small_range_rank[i]
        player_or_manager = 'manager' if 'Manager' in cat else 'player'
        question = f'Which of the following {player_or_manager}s is ranked #{rank} in the {leader_name} leaderboard for \'{stat} ({cat})\'?'
        # get one player that has the correct rank
        correct = df[df['Rank'] == (rank)][name_column].tolist()[0]
        
        # generate wrong answers
        subselect = random.randrange(wrong_count + 1, wrong_count * 2)
        options = random.sample(middle_range_rank, k=min(subselect, len(middle_range_rank)))
        options = [option for option in options if option != rank][:wrong_count]
        wrongs = [df[df['Rank'] == (rank)][name_column].tolist()[0] for rank in options]
        
        # check whether the wrong answer is appropriate or not
        screen_flag = False
        for wrong in wrongs:
            # all the ranks of this player
            ranks_this_player = df[df[name_column].str.contains(clean_name(wrong))]['Rank'].tolist()
            if rank in ranks_this_player:
                screen_flag = True
                break
        if screen_flag:
            continue
        
        # if there is repeating names
        all_options = [clean_name(wrong) for wrong in [correct] + wrongs]
        if len(set(all_options)) != len(all_options):
            continue
            
        tags = [ranking_tag, player_or_manager]
        question_list.append([question] + all_options + [tags])
        
        
    # what is the highest rank of the specific player in the leaderboard
    for i in range(small_range):
        rank = small_range_rank[i]
        player_or_manager = 'manager' if 'Manager' in cat else 'player'
        player_name = df[df['Rank'] == (rank)][name_column].tolist()[0]
        question = f'What is the highest ranking of {clean_name(player_name)}' + \
            f' in the {leader_name} leaderboard for \'{stat} ({cat})\'?'
        
        # find the highest rank of this player
        highest_rank_this_player = min(df[df[name_column].str.contains(clean_name(player_name))]['Rank'].tolist())
        # correct rank
        correct = rank
        
        if correct != highest_rank_this_player:
            continue
        
        # wrong rank
        subselect = random.randrange(wrong_count + 1, wrong_count * 2)
        options = random.sample(middle_range_rank, k=min(subselect, len(middle_range_rank)))
        options = [option for option in options if option != rank][:wrong_count]
        
        all_options = [correct] + options
        if len(set(all_options)) != len(all_options):
            continue
            
        tags = [ranking_tag, player_or_manager]
        question_list.append([question] + all_options + [tags])
    
    # who is ranked highest of the following palyers
    for i in range(small_range):
        rank = small_range_rank[i]
        player_or_manager = 'manager' if 'Manager' in cat else 'player'
        player_name = df[df['Rank'] == (rank)][name_column].tolist()[0]
        question = f'Which of the following {player_or_manager}s ranks the highest' + \
            f' in the {leader_name} leaderboard for \'{stat} ({cat})\'?'
        correct = player_name
        
        # get the number of wrongs to select
        subselect = random.randrange(wrong_count + 1, wrong_count * 2)
        # select ranks that are bigger than rank
        options = random.sample(large_range_rank[(i + 1):], k=min(subselect, len(large_range_rank[(i + 1):])))
        options = [option for option in options if option != rank][:wrong_count]
        wrongs = [df[df['Rank'] == (rank)][name_column].tolist()[0] for rank in options]
        
        all_options = [clean_name(wrong) for wrong in [correct] + wrongs]
        
        # get highest for each player
        #print([correct] + wrongs)
        highest_each_player = [min(df[df[name_column].str.contains(clean_name(player_name))]['Rank'].tolist()) 
                               for player_name in all_options]
        zipped = list(zip(all_options, highest_each_player))
        zipped.sort(key=lambda x: x[1])
        if len(set(all_options)) != len(all_options):
            continue
            
        tags = [ranking_tag, player_or_manager]
        question_list.append([question] + [opt for opt, rk in zipped] + [tags])

100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [00:11<00:00, 15.18it/s]


In [101]:
single_season_ranking_questions = pd.DataFrame(question_list, columns=columns)

In [108]:
with open('questions_single_season_ranking.pkl', 'wb') as f:
    pickle.dump(single_season_ranking_questions, f)
with open('questions_single_season_ranking.pkl', 'rb') as f:
    single_season_ranking_questions = pickle.load(f)

In [109]:
len(single_season_ranking_questions)

2138

In [110]:
print(single_season_ranking_questions.iloc[-824, :].tolist())

["What is the highest ranking of Frank Fennelly in the single-season leaderboard for 'Errors Committed as SS (Fielding)'?", 2, 13, 5, 9, 17, ['Single-season Ranking', 'player']]
